# Assignment 1: Transformations


* TEAM-ID:   25
* TEAM-NAME: 2fast2furious
* YOUR-ID:   
* YOUR-NAME: 

(Although you work in groups, both the students have to submit to Moodle, hence there's name field above)

# Instructions
- Please check Moodle for "TEAM-ID" and "TEAM-NAME" fields above. Some of your names have been edited because of redundancy/simplicity. Instructions for submitting the assignment through GitHub Classrooms/Moodle has been uploaded on Moodle. Any clarifications will be made there itself.
- Code must be written in Python in Jupyter Notebooks. We highly recommend using anaconda distribution or at the minimum, virtual environments for this assignment. See `./misc/installation` for detailed step-by-step instructions about the installation setup.
- Both the team members must submit the zip file.
- For this assignment, you will be using Open3D extensively. Refer to [Open3D Documentation](http://www.open3d.org/docs/release/): you can use the in-built methods and **unless explicitly mentioned**, don't need to code from scratch for this assignment. Make sure your code is modular since you will be reusing them for future assignments.
- Take a look at the entire assignment. The descriptive questions at the end might have a clue if you are stuck somewhere.
- Answer the descriptive questions in your own words with context & clarity. Do not just copy-paste from some Wikipedia page. You will be evaluated accordingly.
- Please call the visualization functions only when they are asked. They are asked explicitly at the end of every section.
- You could split the Jupyter Notebook cells where `TODO` is written, but please try to avoid splitting/changing the structure of other cells.

In [1]:
import open3d as o3d
import copy
import numpy as np
from scipy.linalg import logm
from scipy.spatial.transform import Rotation
from scipy.optimize import fsolve

# 1. Getting started with Open3D
## 1.1 Converting RGBD image into Point Cloud


In your robotics journey, it is common to be given just the depth images along with camera parameters in a generic dataset and you'd want to build a 3D data representation out of it, for example, a point cloud. You will understand the math behind these concepts in detail during Vision classes, for now, you can use the in-built functions as a black box.
- Below are the given RGB and D images from SUN RGB-D dataset ([S.Song, CVPR 2015](https://rgbd.cs.princeton.edu/)). 
![low-res-RGBD.resized.png](./misc/low-res-RGBD.png)

- Read these two images `color.jpg` and `depth.png` given in current folder using Open3D. Convert it to a point cloud using the default camera parameters (`o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault`).
Then,
- Create a "world" frame $A$ and combine this (just use $+$ operator) with the above point cloud and save it as `scene.pcd`. Put it aside for now.
- Write a simple function `one_one` to visualize `scene.pcd`.

In [2]:
def mesh2pcd(triangle_mesh):
    pcd = o3d.geometry.PointCloud()
    pcd.points = triangle_mesh.vertices
    pcd.colors = triangle_mesh.vertex_colors
    pcd.normals = triangle_mesh.vertex_normals
    return pcd

def custom_draw_geometry(pcd, edit=False):
    # The following code achieves the same effect as:
    # o3d.visualization.draw_geometries([pcd])
    if edit:
        vis = o3d.visualization.VisualizerWithEditing()
    else:
        vis = o3d.visualization.Visualizer()
    vis.create_window(width=1280, height=720)
    vis.add_geometry(pcd)
    vis.run()
    vis.destroy_window()
    if edit:
        return vis.get_picked_points()
    else:
        return None


In [3]:
##############################################################################
# TODO: Do tasks described in 1.1                                            #
##############################################################################

ORIGIN_frameA = [5, -3, 5]

color = o3d.io.read_image('color.jpg')
depth = o3d.io.read_image('depth.png')
rgbd_image = o3d.geometry.RGBDImage.create_from_sun_format(color, depth)

pcd = o3d.geometry.PointCloud.create_from_rgbd_image(
        rgbd_image,
        o3d.camera.PinholeCameraIntrinsic(o3d.camera.PinholeCameraIntrinsicParameters.PrimeSenseDefault))

pcd.transform([[1, 0, 0, 0], [0, -1, 0, 0], [0, 0, -1, 0], [0, 0, 0, 1]])

mesh_frameA = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=0.6, origin=ORIGIN_frameA)

pcd_coordinateFrameA = mesh2pcd(mesh_frameA)

pcd = pcd + pcd_coordinateFrameA
# o3d.visualization.draw_geometries([pcd])
o3d.io.write_point_cloud("scene.pcd", pcd)

def one_one(pcd_path = 'scene.pcd'):
    pcd = o3d.io.read_point_cloud(pcd_path)
    custom_draw_geometry(pcd)

##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

### Question for 1.1

- In the next code cell, call the function `one_one` here showing `scene.pcd`.

In [4]:
#uncomment the following and add input parameters if any
one_one()

# 2. Rotations, Euler angles and Gimbal Lock

## 2.1 Rotating an object

The objective here is to roughly simulate an object moving on a ground.

- Generate a cube at some point on the ground and create another frame $B$ at the center of this object. Combine these both as a single point cloud `cube.pcd`. (You can pick a point on the ground by using the `get_picked_points` method of the class `open3d.visualization.VisualizerWithEditing`.)
- Now read both the point clouds `scene.pcd` and `cube.pcd` in a script. Whatever tasks you do below are on the object `cube.pcd` (along with the axes $B$) with `scene.pcd` in the background (static).
- Given a sequence of **ZYX Euler** angles $[30^{\circ}, 90^{\circ}, 45^{\circ}]$, generate the rotation. In our case, our object (with its respective axis) undergoes rotation with the background being fixed (with its respective axis).
- Note: Throughout this assignment, we will be using the standard **ZYX** Euler angle convention.
- Write a function `two_one` to show the above by **animation** (cube rotating along each axis one by one).
    - *Hint: Use Open3D's non-blocking visualization and discretize the rotation to simulate the animation. For example, if you want to rotate by $30^{\circ}$ around a particular axis, do in increments of $5^{\circ}$ 6 times to make it look like an animation.*

In [5]:
def mesh2pcd_test(triangle_mesh):
    pcd = triangle_mesh.sample_points_uniformly(number_of_points = 5000)
    return pcd

In [6]:
##############################################################################
# TODO: Do tasks described in 2.1                                            #
##############################################################################
# Replace "pass" statement with your code


# mesh_box = o3d.geometry.TriangleMesh.create_box(width=1.0, height=1.0, depth=1.0)
# pcd_box = mesh2pcd(mesh_box)

######   Create Cube  ######
vec_len = 1.5
mesh_arrow = o3d.geometry.TriangleMesh.create_arrow(
        cone_height= 0.2 * vec_len, 
        cone_radius= 0.06 * vec_len, 
        cylinder_height= 0.8 * vec_len,
        cylinder_radius=  0.04 * vec_len
        )
mesh_arrow.paint_uniform_color([1,0,1])
mesh_arrow.compute_vertex_normals()
pcd_arrow = mesh2pcd(mesh_arrow)

x = np.linspace(-0.5, 0.5, 10)
mesh_x, mesh_y, mesh_z = np.meshgrid(x, x, x)
xyz = np.zeros((np.size(mesh_x), 3))
xyz[:, 0] = np.reshape(mesh_x, -1)
xyz[:, 1] = np.reshape(mesh_y, -1)
xyz[:, 2] = np.reshape(mesh_z, -1)
pcd_box = o3d.geometry.PointCloud()
pcd_box.points = o3d.utility.Vector3dVector(xyz)

ORIGIN_frameB = pcd_box.get_center()
mesh_frameB = o3d.geometry.TriangleMesh.create_coordinate_frame(
        size=1, origin=ORIGIN_frameB)

mesh_frameB.paint_uniform_color([0.5,0.3,1])
pcd_coordinateFrameB = mesh2pcd(mesh_frameB)

scene_pcd = o3d.io.read_point_cloud('scene.pcd')
# ORIGIN_B = custom_draw_geometry(scene_pcd, True)
ORIGIN_B = [218821]
# print(ORIGIN_B)
assert (len(np.asarray(scene_pcd.points)[ORIGIN_B] >= 3))

ORIGIN_B = np.asarray(scene_pcd.points)[ORIGIN_B][0]

cube = pcd_box + pcd_coordinateFrameB + pcd_arrow
cube.translate(ORIGIN_B)
o3d.io.write_point_cloud("cube.pcd", cube)

# R = cube.get_rotation_matrix_from_zyx(np.array([np.pi/6,np.pi/2,np.pi/4]))

# cube.rotate(R, center = ORIGIN_frameB)

cube_pcd = o3d.io.read_point_cloud('cube.pcd')

def two_one(cube_pcd = o3d.io.read_point_cloud('cube.pcd'), scene_pcd = o3d.io.read_point_cloud('scene.pcd'), 
            center = ORIGIN_frameB,
            angles_zyx = np.array([np.pi/6, np.pi/2, np.pi/4]), 
            increment = 0.1*(np.pi/180)):
    
    frames_per_angle = (angles_zyx/increment).astype('int')
    
    rot_z = np.cumsum(np.tile(np.array([increment,0,0]),(frames_per_angle[0],1)),axis = 0)
    rot_y = np.cumsum(np.tile(np.array([0,increment,0]),(frames_per_angle[1],1)),axis = 0) + np.tile(np.array([angles_zyx[0],0,0]),(frames_per_angle[1],1))
    rot_x = np.cumsum(np.tile(np.array([0,0,increment]),(frames_per_angle[2],1)),axis = 0) + np.tile(np.array([angles_zyx[0],angles_zyx[1],0]),(frames_per_angle[2],1))
    
    rot_list = np.vstack((rot_z,rot_y,rot_x))
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(width = 1280, height = 720)
    vis.add_geometry(cube_pcd)
    vis.add_geometry(scene_pcd)
    prev_angle = np.zeros(3)
    
    for idx, angle in enumerate(rot_list):
        
        R_prev = cube_pcd.get_rotation_matrix_from_zyx(prev_angle)
        R = cube_pcd.get_rotation_matrix_from_zyx(angle)
        prev_angle = angle
        cube_pcd.rotate(R_prev.T,center = ORIGIN_B)
        cube_pcd.rotate(R,center = ORIGIN_B)

        vis.update_geometry(cube_pcd)
        
        vis.poll_events()
        vis.update_renderer()
    vis.run() 
    vis.destroy_window()
    del vis
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################


### Question for 2.1

- In the next code cell, call the function `two_one` here showing the animation described in section 2.1.

In [7]:
#uncomment the following and add input parameters if any
two_one(increment = 0.04*(np.pi/180))

## 2.2 Euler angle & Gimbal lock

Code the following yourself from scratch (Refer Craig book - Section: $Z-Y-X$ Euler angles - same conventions/notations followed).

- Case 1: Given the rotation matrix $M_{given}$ below, extract Euler angles $\alpha , \beta ,\gamma$. Convert it back to the rotation matrix $M_{recovered}$ from Euler angles.

    $$M(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0.26200263 & -0.19674724 & 0.944799 \\0.21984631 & 0.96542533 & 0.14007684 \\
    -0.93969262 & 0.17101007 & 0.29619813\end{array}\right] $$

    After coding it from scratch, check your calculations using `scipy.spatial.transform.Rotation`. (Mandatory)

- Case 2: Given the rotation matrix $N_{given}$, extract Euler angles, and convert back $N_{recovered}$.

    $$N(\alpha , \beta ,\gamma)=\left[\begin{array}{rrr}0 & -0.173648178 &  0.984807753 \\0 & 0.984807753 & 0.173648178 \\
    -1 & 0 & 0\end{array}\right] $$

    Again use `scipy` and check its output. If `scipy` is showing any warnings on any of the above cases, explain it in "**Questions for 2.2**" (last question). Write code in the next cell.
    
- (Optional) Case 3: Do the above two for quaternion using scipy functions, i.e. given the rotation matrix, extract quaternion and convert back.

In [8]:
##############################################################################
# DON'T EDIT
M_given =  np.array([[0.26200263, -0.19674724, 0.944799],
                     [0.21984631, 0.96542533, 0.14007684],
                     [-0.93969262, 0.17101007, 0.29619813]])

N_given = np.array([[0,-0.173648178,0.984807753],
                    [0, 0.984807753, 0.173648178],
                    [-1, 0, 0]])

In [9]:
# TODO: Do tasks described in 2.2                                            #
##############################################################################
# Replace "pass" statement with your code

def rotm2eul(R):
    beta = np.arctan2(-R[2,0], np.sqrt(R[0,0]**2 + R[1,0]**2))
    alpha = np.arctan2(R[1,0]/np.cos(beta), R[0,0]/np.cos(beta))
    gamma = np.arctan2(R[2,1]/np.cos(beta), R[2,2]/np.cos(beta))
    
    return np.array([alpha,beta,gamma])

def eul2rotm(angles_zyx):
    a = angles_zyx[0]; b = angles_zyx[1]; g = angles_zyx[2]
    
    R = np.array([
        [np.cos(a)*np.cos(b), np.cos(a)*np.sin(b)*np.sin(g)-np.sin(a)*np.cos(g), np.cos(a)*np.sin(b)*np.cos(g)+np.sin(a)*np.sin(g)],
        [np.sin(a)*np.cos(b), np.sin(a)*np.sin(b)*np.sin(g)+np.cos(a)*np.cos(g), np.sin(a)*np.sin(b)*np.cos(g)-np.cos(a)*np.sin(g)],
        [-np.sin(b), np.cos(b)*np.sin(g), np.cos(b)*np.cos(g)]
    ])
    
    return R

### Case 1 ###
print('Case 1\n')

M_Euler_angles = rotm2eul(M_given)
print('Euler Angles: ', M_Euler_angles)

M_recovered = eul2rotm(M_Euler_angles)
print('\nM_recovered: \n',M_recovered)


M_Euler_scipy = Rotation.from_matrix(M_given).as_euler('ZYX')
print('\nEuler Angles from scipy: ',M_Euler_scipy)
print('\nDiffrence in angles wrt scipy: ', M_Euler_angles - M_Euler_scipy)

M_recovered_scipy = Rotation.from_euler('ZYX', M_Euler_scipy).as_matrix()
print('\nM_recovered from scipy: \n',M_recovered_scipy)
print('\nDifference in recovered matrices: \n', M_recovered - M_recovered_scipy)

### Case 2 ###
print('\n\nCase 2 \n')

N_Euler_angles = rotm2eul(N_given)
print('Euler Angles: ', N_Euler_angles)

N_recovered = eul2rotm(N_Euler_angles)
print('\nN_recovered: \n',N_recovered)


N_Euler_scipy = Rotation.from_matrix(N_given).as_euler('ZYX')
print('\nEuler Angles from scipy: ',N_Euler_scipy)
print('\nDiffrence in angles wrt scipy: ', N_Euler_angles - N_Euler_scipy)

N_recovered_scipy = Rotation.from_euler('ZYX', N_Euler_scipy).as_matrix()
print('\nN_recovered from scipy: \n',N_recovered_scipy)
print('\nDifference in recovered matrices: \n', N_recovered - N_recovered_scipy)


### Case 3 ###
print('\n\nCase 3 \n')

M_quat_scipy = Rotation.from_matrix(M_given).as_quat()
print('\nM_Quaternion from scipy: ',M_quat_scipy)

M_recovered_quat_scipy = Rotation.from_quat(M_quat_scipy).as_matrix()
print('\nM_recovered from scipy: \n',M_recovered_quat_scipy)

N_quat_scipy = Rotation.from_matrix(N_given).as_quat()
print('\nN_Quaternion from scipy: ',N_quat_scipy)

N_recovered_quat_scipy = Rotation.from_quat(N_quat_scipy).as_matrix()
print('\nN_recovered from scipy: \n',N_recovered_quat_scipy)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

Case 1

Euler Angles:  [0.6981317  1.22173048 0.52359878]

M_recovered: 
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

Euler Angles from scipy:  [0.69813171 1.22173048 0.52359878]

Diffrence in angles wrt scipy:  [-5.33206246e-09 -3.56592400e-09 -8.31095359e-09]

M_recovered from scipy: 
 [[ 0.26200263 -0.19674724  0.944799  ]
 [ 0.21984631  0.96542534  0.14007684]
 [-0.93969262  0.17101007  0.29619813]]

Difference in recovered matrices: 
 [[ 3.73915132e-09 -3.17161308e-09 -1.69737013e-09]
 [ 7.56884888e-10 -5.07081044e-10  2.30695280e-09]
 [ 1.21961785e-09 -7.86252702e-10  4.32319730e-09]]


Case 2 

Euler Angles:  [0.         1.57079633 0.        ]

N_recovered: 
 [[ 6.123234e-17  0.000000e+00  1.000000e+00]
 [ 0.000000e+00  1.000000e+00  0.000000e+00]
 [-1.000000e+00  0.000000e+00  6.123234e-17]]

Euler Angles from scipy:  [0.17453293 1.57079633 0.        ]

Diffrence in angles wrt scipy:  [-0.17453293  0.   

/home/soumyasis/anaconda3/envs/open3d/lib/python3.6/site-packages/scipy/spatial/transform/rotation.py:133: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  warnings.warn("Gimbal lock detected. Setting third angle to zero since"


## Questions for 2.2
- Have you used `np.arctan` or an any equivalent `atan` function above? Why or why not?   
    * Ans: *Yes, we have used np.arctan2 to determine uniquely the euler angles for rotation matrix because it also stores information about the quadrant the angle belongs to.*

### For Case 1 above,
- What Euler angles  $\alpha , \beta ,\gamma$ did you get? Replace `my_array_case1` with your array.

In [10]:
# Uncomment and replace my_array_case1 with your array.

print("My Euler angles for case 1 are " + str(M_Euler_angles))

My Euler angles for case 1 are [0.6981317  1.22173048 0.52359878]


  - Were you able to recover back your rotation matrix when you converted it from Euler angles? Why/why not? Replace `M_given` and `M_recovered` with your matrices below and explain "why/why not" after this code snippet.

In [11]:
# Uncomment and Replace M_given and M_recovered with your matrices below.
error = np.linalg.norm(logm(M_given @ M_recovered.T))
print("For case 1, it is " + str(error<0.0001) + " I could recover the original matrix.")

For case 1, it is True I could recover the original matrix.


- Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 1? If yes, what is it?

    - Ans:    *No Gimbal Lock is present so euler angle rotation can work.*

### Repeat the above for Case 2.

In [12]:
# Uncomment and Replace N_given and N_recovered with your matrices below.
print("My Euler angles for case 2 are " + str(N_Euler_angles))
error = np.linalg.norm(logm(N_given @ N_recovered.T))
print("For case 2, it is " + str(error<0.0001) + " I could recover the original matrix.")

My Euler angles for case 2 are [0.         1.57079633 0.        ]
For case 2, it is False I could recover the original matrix.


* Why/why not? Based on your observations here, is there any problem with Euler angle representation for Case 2? If yes, what is it?

    * Ans: *Here Gimbal Lock is present. So in Euler representation rotation about z and x axis coincide.*
    
    
* Explain any more problems with Euler angle representation. Explain what you understand by Gimbal lock (concisely in your own words). You could revisit this question in the section 2.4.
    * Ans: *Quaternion is a 4D representation which represents 3D rotation, that’s why it is sufficient to avoid any ambiguities, while euler is 3D representation and the gimbal lock ambiguity cannot be removed in this representation.*
    * *Gimbal lock is the loss of one degree of freedom in a three-dimensional space that occurs when the axes of two of the three gimbals are driven into a parallel configuration, "locking" the system into rotation in a degenerate two-dimensional space.*

- When you used `scipy.spatial.transform.Rotation` for the above 2 cases,
    - Have you used `zyx` above in `r.as_euler('')` argument? Why or why not? Explain the difference between extrinsic and instrinsic rotations with equivalent technical names from Craig book?
        * Ans: *No we used 'ZYX' notation which is Intrinsic Rotation. Extrinsic means the original coordinate system remains motionless as the rotations go. Intrinsic means the rotation is always based on the rotating coordinate system which is solidary with the moving object.*
    - Has `scipy` shown any warnings on any of the above cases? If yes, explain it.
        * Ans: *For Case 2, Scipy shows Gimbal Lock Warning because $\beta$ is 90 degrees*
    - (Optional) For Case 3 above (quaternion) which you did using scipy, did you observe any problem with quaternion? Depending on your observations, which is better? Quaternion or Euler angles? And why?
         * Ans: *No we didn't observe any problem with quarternion. Quarternion is better because it is a 4D representation which represents 3D rotation, that’s why it is sufficient to avoid any ambiguities, while euler is 3D representation and the gimbal lock ambiguity cannot be removed in this representation.*


## 2.3 Rotation matrix as an Operator
This question will help you in your understanding of [Rotator-transform (Vector-frame) equivalence](https://www.notion.so/saishubodh/Lecture-2-Transformations-11d69d8cef2d4cd195a98fa7d33224e1#f90ece4f5e374743bfed47e46a83ecfe).

![image.png](./misc/xyz-frame.png)
Consider the frame $XYZ$ in the above image. Say you have a vector $x_1=[0,\sqrt{3},0]$. Now you want to rotate it such that you end up at $x_2=[1,1,1]$ through a sequence of Euler angle rotations. Your goal is to find out those $\alpha, \beta \: \& \: \gamma$ ($ZYX$). We will follow this order whenever we refer to it below.

First, properly understand the so-called "Rotator-transform equivalence" to figure out what are the terms of rotation matrix. Then, put the math on paper and you will end up with a set of non-linear equations. Write the set of linear equations in LaTeX here:

$$ \text{Ans: For two vectors $\overrightarrow{\textbf{x}_{1}}$ and $\overrightarrow{\textbf{x}_{2}}$}\newline
\begin{bmatrix} 
cos(\alpha)cos(\beta) & cos(\alpha)sin(\beta)sin(\gamma)-sin(\alpha)cos(\gamma) & cos(\alpha)sin(\beta)cos(\gamma)-sin(\alpha)sin(\gamma) \\
sin(\alpha)cos(\beta) & sin(\alpha)sin(\beta)sin(\gamma)-cos(\alpha)cos(\gamma) & sin(\alpha)sin(\beta)cos(\gamma)-cos(\alpha)sin(\gamma)\\
-sin(\beta) & cos(\beta)sin(\gamma) & cos(\beta)cos(\gamma) \\
\end{bmatrix}
\begin{bmatrix}
x_{11}\\
x_{12}\\
x_{13}
\end{bmatrix} - \begin{bmatrix}
x_{21}\\
x_{22}\\
x_{23}
\end{bmatrix} = 
\begin{bmatrix}
0\\
0\\
0
\end{bmatrix}$$

Solve these equations using `fsolve` from `scipy.optimize` as follows: (Come back and answer the following questions after coding it in the next block)
- `case1`: First, solve it with an initialization of (0,0,0). Check if your answer is correct using `np.isclose`.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_1, b_1, c_1$ = your answer here.* 
- `case2`: Now, forget about the solver for a moment: Can you visualize and think of sequence of rotations one by one to reach the final position (which is different than previous set of rotations)? Now, validate your answer by giving (your answer $\pm 5$) as initialization.
    * What Euler angles did you get? Answer in $\alpha, \beta \: \& \: \gamma$ format:
        * Ans: *$a_2, b_2, c_2$ = your answer here.*

In [13]:
##############################################################################
# DON'T EDIT
x_1 = np.array([0,np.sqrt(3),0])
x_2 = np.array([1,1,1])

In [14]:
##############################################################################
# TODO: Do tasks described in 2.3                                            #
##############################################################################
# Replace "pass" statement with your code

def func(angles_zyx):
    return np.squeeze(np.matmul(eul2rotm(angles_zyx),x_1[:,None]) - np.array(x_2[:,None]))

a1,b1,c1 = fsolve(func, [0.,0.,0.])
a2,b2,c2 = fsolve(func, [-np.pi/4 ,0, np.arctan2(1,np.sqrt(2))])
print('a1 ,b1, c1 = ',a1, b1, c1)
print('a2 ,b2, c2 = ',a2, b2, c2)
##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

a1 ,b1, c1 =  -0.32906483821879123 -93.69048524478949 0.7481509761190938
a2 ,b2, c2 =  -0.7853981633974483 0.0 0.6154797086703873


In [15]:
# From Section 2.2, use the function which takes Euler angles and gives Rotation matrix as output.
# Uncomment and replace `r_mat` with the name of the function. (Do NOT edit anything else)

x_2_obtained_case1 = eul2rotm([a1,b1,c1]) @ x_1.T #TODO: replace r_mat
x_2_obtained_case2 = eul2rotm([a2,b2,c2]) @ x_1.T #TODO: replace r_mat
test = True #TODO: Set this as True

In [16]:
# DON'T EDIT
if test == True:
    case1_test = np.isclose(x_2_obtained_case1,  np.array([1.0,1.0,1.0]))
    case2_test = np.isclose(x_2_obtained_case2,  np.array([1.0,1.0,1.0]))
    print("For case 1, it is " + str(bool(case1_test[0] and case1_test[1] and case1_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")
    print("For case 2, it is " + str(bool(case2_test[0] and case2_test[1] and case2_test[2])) + 
          " that I could end up at (1,1,1) after rotation.")

For case 1, it is True that I could end up at (1,1,1) after rotation.
For case 2, it is True that I could end up at (1,1,1) after rotation.


## 2.4 Gimbal Lock visualization (Optional)

A nice visualization video for gimbal lock is [this](https://www.youtube.com/watch?v=zc8b2Jo7mno). You are about to animate a similar visualization demonstrating gimbal lock 😃.

![image.png](./misc/xyz-frame.png)

- Write a function `two_four` for the visualization of gimbal lock. Follow the below steps to get the intuition going. Use Open3D for the following.
    - Say our frame's initial position is as the above image. Now the final goal at the end of rotation is to get the $Y$ axis pointing in the direction of the vector $(x,y,z)$ that you currently see in the above image. This point is fixed in space and is NOT moving as we rotate our axis.
    - For creating that point, you could use a small sphere using `open3d.geometry.create_mesh_sphere`. You already know how to create an axis by now.
    - Following our $ZYX$ convention, first rotate your frame about $Z$ axis by an angle, say $-35^{\circ}$. Then rotate about $Y$ axis by an angle ${\beta}$ and then about $X$ by say $55^{\circ}$.
        - Are there any specific angle(s) $\beta$ using which you will **never** reach our point $(x,y,z)$ ?
            - Clue: We are specifically talking about gimbal lock here and notice the word "never".
        - Under this (these) specific angle(s) of $\beta$ & different combinations of $\alpha$ and $\gamma$, make an animation and clearly show why your $Y$ axis is unable to align in the direction of that vector $(x,y,z)$ using the animation.

            If you are unsure to simulate the animation, you could do it as follows:

            - You could first fix some $\alpha$, say $-35^{\circ}$ & an above value of $\beta$, you can now vary $\gamma$ from $-180^{\circ} \text{ to }180^{\circ}$ to simulate the animation.
            - Now fix another $\alpha$, say $45^{\circ}$ and repeat the above process. So that's 2 specific values of $\alpha$.
            - Show this for all angles of $\beta$ if there are more than 1.
            - Therefore, when the code is run, there should be a minimum of (2 $\times$ (number of values of $\beta$)) animations. 2 for values of $\alpha$, you could show it for even more if you wish to.

In [17]:
##############################################################################
# TODO: Do tasks described in 2.4                                            #
##############################################################################
# Replace "pass" statement with your code

def two_four(alpha = [0,-35], beta = [0,90], gamma = [0,360], degrees = True, increment = 0.5):
    
    ORIGIN = [0,0,0]
    point_loc = [1,1,1]

    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=np.sqrt(3), origin=ORIGIN)
    mesh_frame1 = o3d.geometry.TriangleMesh.create_coordinate_frame(
            size=np.sqrt(3), origin=ORIGIN)
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius = 0.08).translate(point_loc)
    mesh_sphere1 = o3d.geometry.TriangleMesh.create_sphere(radius = 0.08).translate([0,np.sqrt(3),0])
    mesh_sphere2 = o3d.geometry.TriangleMesh.create_sphere(radius = 0.08).translate([np.sqrt(1.5),np.sqrt(1.5),0])
    mesh_sphere3 = o3d.geometry.TriangleMesh.create_sphere(radius = 0.08).translate([0,np.sqrt(2),1])
    
    mesh_sphere.paint_uniform_color([0.5,0.3,1])
    mesh_frame1.paint_uniform_color([0.5,0.3,1])

    if(degrees):
        alpha_range = np.array(alpha)*(np.pi/180)
        beta_range = np.array(beta)*(np.pi/180)
        gamma_range = np.array(gamma)*(np.pi/180)
    else:
        alpha_range = np.array(alpha)
        beta_range = np.array(beta)
    #     gamma_range = np.array([0,np.arctan2(1,np.sqrt(2))])
        gamma_range = np.array(gamma)

    R_init = mesh_frame.get_rotation_matrix_from_zyx((alpha_range[0],beta_range[0],gamma_range[0]))
    mesh_frame.rotate(R_init, center = ORIGIN)

    angles_zyx = np.vstack((alpha_range,beta_range,gamma_range))
    angles_zyx = angles_zyx[:,1] - angles_zyx[:,0]
    increment = increment*(np.pi/180)

    frames_per_angle = (angles_zyx/increment).astype('int')
    direction = 2*(frames_per_angle >= 0) - 1
    frames_per_angle = np.abs(frames_per_angle)
    
    rot_z = np.cumsum(np.tile(direction[0]*np.array([increment,0,0]),(frames_per_angle[0],1)),axis = 0)
    rot_y = np.cumsum(np.tile(direction[1]*np.array([0,increment,0]),(frames_per_angle[1],1)),axis = 0) + np.tile(np.array([angles_zyx[0],0,0]),(frames_per_angle[1],1))
    rot_x = np.cumsum(np.tile(direction[2]*np.array([0,0,increment]),(frames_per_angle[2],1)),axis = 0) + np.tile(np.array([angles_zyx[0],angles_zyx[1],0]),(frames_per_angle[2],1))
    
    rot_list = np.vstack((rot_z,rot_y,rot_x))
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(width = 1280, height = 720)
    vis.add_geometry(mesh_frame)
    vis.add_geometry(mesh_sphere)
#     vis.add_geometry(mesh_sphere1)
#     vis.add_geometry(mesh_sphere2)
#     vis.add_geometry(mesh_sphere3)
    vis.add_geometry(mesh_frame1)
    prev_angle = np.zeros(3)
    
    for angle in rot_list:
        
        R_prev = mesh_frame.get_rotation_matrix_from_zyx(prev_angle)
        R = mesh_frame.get_rotation_matrix_from_zyx(angle)
        prev_angle = angle
        mesh_frame.rotate(R_prev.T,center = ORIGIN)
        mesh_frame.rotate(R,center = ORIGIN)

        vis.update_geometry(mesh_frame)
        vis.poll_events()
        vis.update_renderer()
    vis.run() 
    vis.destroy_window()





##############################################################################
#                             END OF YOUR CODE                               #
##############################################################################

> VOILA! You have just animated the famous Gimbal lock problem. If you are curious, read about the [Apollo 11](https://en.wikipedia.org/wiki/Gimbal_lock#On_Apollo_11) Gimbal lock incident.

### Questions for 2.4 (Optional)

- Mention the value(s) of $\beta$ here: 
    * Ans: $$\pm\frac{\pi}{2}\:radians$$ 
- Now that you understand gimbal lock through visualization, explain it now in matrix form: For the above values of $\beta$, what does the rotation matrix look like? Can you explain why gimbal lock occurs from the rotation matrix alone? Clue: Use sin/cos formulae. 
    * Ans: *$$ \text{The Rotation matrix for $\beta = \frac{\pi}{2}$ radians will be}\newline
\begin{bmatrix} 
0 & cos(\alpha)sin(\gamma)-sin(\alpha)cos(\gamma) & cos(\alpha)cos(\gamma)-sin(\alpha)sin(\gamma) \\
0 & sin(\alpha)sin(\gamma)-cos(\alpha)cos(\gamma) & sin(\alpha)cos(\gamma)-cos(\alpha)sin(\gamma)\\
-1 & 0 & 0 \\
\end{bmatrix} = \begin{bmatrix} 
0 & sin(\gamma-\alpha) & cos(\alpha+\gamma) \\
0 & -cos(\alpha+\gamma) & sin(\alpha-\gamma)\\
-1 & 0 & 0 \\
\end{bmatrix}\newline$$*
* *This clearly shows that the 3D rotation sequence has degenerated into a rotation in only 2 planes. This is the Gimbal Lock problem*
- Call the function `two_four` for the visualization of gimbal lock written above. 

In [18]:
#uncomment the following and add input parameters if any
two_four([0,-35],[0,90], increment =0.04)
two_four([0,-35],[0,-90], increment =0.04)
# two_four([0,np.pi/4],[0,0],[0,np.arctan2(1,np.sqrt(2))],degrees = False)
two_four([0,45],[0,-90], increment =0.04)
two_four([0,a2],[0,b2],[0,c2], False, increment =0.04)